In [1]:
#Lipid parser
#08/11/2023
#Gerard Baquer
import re
import pandas as pd
import pygoslin
from pygoslin.parser.Parser import LipidParser
from tkinter.filedialog import askopenfilename

#0. Load Data
data_filename=''
fa_filename='../data/fa.csv'
if data_filename=='':
    data_filename = askopenfilename(filetypes=[('.csv', '.csv')],title="Choose input data")

data=pd.read_csv(data_filename)
fa=pd.read_csv(fa_filename)

#1. Define parsers
functionalgroup_re='(-\w+\(\d+[ A-Za-z,\d]*\))|\+=\w\(\d+\)'
lipid_parser = LipidParser()
def tryparse(x):
    try:
        x=re.sub(functionalgroup_re,'',x)
        for i in range(fa.shape[0]):
            x=re.sub(fa.name[i],fa.repname[i],x)
        return(lipid_parser.parse(x))
    except:
        return
        
def format(lipid):
    if(isinstance(lipid,pygoslin.domain.LipidAdduct.LipidAdduct)):
        hg=lipid.lipid.headgroup.get_lipid_string()
        c=sum(list(map(lambda x: x.num_carbon,lipid.lipid.fa_list)))
        db=sum(list(map(lambda x: x.get_double_bonds(),lipid.lipid.fa_list)))
        return(hg+" "+str(c)+":"+str(db))

#2. Parse
lipids=list(map(lambda x : tryparse(x), data.compound_name))
data['newname']=list(map(format,lipids))
data['functionalgroups']=list(map(lambda x: str(re.findall(functionalgroup_re,x)),data.compound_name))
data['sidechains']=list(map(lambda x: str(re.findall("|".join(fa.name),x)),data.compound_name))

#3. Export
out_filname=re.sub(".csv","_out.csv",data_filename)
data.to_csv(out_filname)
print("You are all set!")
print(out_filname)

You are all set!
A:/59. Lipid Parsing/data_out.csv
